In [8]:
!pip install -q openai
!pip install langchain langchain-openai -q
!pip install faiss-cpu -q
!pip install langchain_community -q
!pip install tiktoken -q

In [9]:
from openai import OpenAI

## Set the API key and model name
MODEL="gpt-4o-mini-2024-07-18"
client = OpenAI(api_key="sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd")

documents.txt

In [10]:
# Define the content for both basic and advanced sections
content = """
문서 1:
북극곰은 바다 얼음에 의존하여 물범을 사냥하기 때문에 기후 변화에 직접적인 영향을 받고 있습니다. 지구 온난화로 인해 바다 얼음이 매년 더 일찍 녹고 늦게 형성되면서,
북극곰이 사냥할 수 있는 시간이 줄어들고 있습니다. 이는 북극곰의 신체 상태 악화, 새끼의 생존율 저하, 그리고 일부 경우에는 사망률 증가로 이어졌습니다.

문서 2:
해빙 서식지의 상실은 북극곰 개체군에 가장 큰 위협 중 하나입니다. 얼음이 녹으면서 북극곰은 먹이를 찾기 위해 더 먼 거리를 이동해야 하며 이로 인해 에너지 소모가 증가합니다.
이로 인해 영양실조와 생식 성공률이 감소하여 종을 더욱 위험에 빠뜨릴 수 있습니다.

문서 3:
기후 변화는 북극곰의 사냥터뿐만 아니라 보금자리 구역에도 영향을 미치고 있습니다. 더 따뜻한 기온과 불안정한 눈 조건으로 인해
임신한 암컷이 적합한 보금자리 지역을 찾는 것이 더 어려워지고 있으며, 이는 새끼의 출생과 생존에 매우 중요합니다.
이는 바다 얼음 손실로 이미 어려움을 겪고 있는 북극곰 개체군에 추가적인 위험 요소가 되고 있습니다.

문서 4:
전통적인 식량 공급원이 점점 더 어려워지면서 일부 북극곰은 인간의 배설물을 먹거나 바다새와 그 알을 잡아먹는 모습이 관찰되었습니다.
이러한 행동은 단기적인 안도감을 제공할 수 있지만, 혹독한 북극 환경에서 장기적으로 생존하는 데 필수적인 물개로부터 얻는 고지방 식단을 대체하지는 못합니다.

문서 5:
최근 연구에 따르면 온실 가스 배출의 현재 추세가 계속된다면 북극곰은 이 세기 안에 멸종에 직면할 수 있습니다.
보존 노력은 전 세계 배출량을 줄이고 중요한 북극곰 서식지를 보호하는 데 집중하고 있지만 기후가 현재 속도로 계속 따뜻해진다면 이러한 노력만으로는 충분하지 않을 수 있습니다.
"""

# Write the content to a file
with open('documents.txt', 'w') as file:
    file.write(content)

In [6]:
!ls

documents.txt  sample_data


In [11]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

# OpenAI API 키 설정
import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd'  # OpenAI API 키 입력

# 1. 문서 로드
# 텍스트 파일을 사용해 문서를 로드합니다.
loader = TextLoader('documents.txt')
documents = loader.load()

# 2. 문서 임베딩 및 인덱싱
# 문서를 벡터로 변환한 후, FAISS 벡터 저장소에 저장합니다.
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# 3. Fine-tuned GPT 모델을 사용한 LLM 생성
# Fine-tuned된 GPT 모델을 사용할 수 있도록 설정합니다.
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)

# 4. 문서 검색 개수 조정 (최대 5개의 문서 검색)
# 검색할 문서의 개수를 5개로 설정하여 모든 문서가 context에 반영될 수 있도록 설정합니다.
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# 5. RAG 구현
# RetrievalQA 체인을 사용해 검색된 문서를 기반으로 LLM을 통해 답변을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # Fine-tuned된 GPT 모델.
    chain_type="stuff",  # 'stuff' 체인을 사용하여 검색된 문서들을 단일 입력으로 결합하여 처리.
    retriever=retriever,  # 검색할 문서의 수를 5개로 설정한 retriever.
    return_source_documents=True  # 검색된 문서를 포함하여 반환할지 여부를 지정
)

# 6. 질문에 대한 답변 생성
query = "기후 변화가 북극곰에 어떤 영향을 미치는가?"
result = qa_chain.invoke({"query": query})  # invoke 메서드를 사용하여 query 전달

# 7. 결과 출력
print(result["result"])

기후 변화는 북극곰에 여러 가지 부정적인 영향을 미치고 있습니다. 첫째, 북극곰은 바다 얼음에 의존하여 물범을 사냥하는데, 지구 온난화로 인해 바다 얼음이 매년 더 일찍 녹고 늦게 형성되면서 사냥할 수 있는 시간이 줄어들고 있습니다. 이로 인해 북극곰의 신체 상태가 악화되고, 새끼의 생존율이 저하되며, 사망률이 증가하는 문제가 발생하고 있습니다.

둘째, 해빙 서식지의 상실로 인해 북극곰은 먹이를 찾기 위해 더 먼 거리를 이동해야 하며, 이는 에너지 소모를 증가시킵니다. 이로 인해 영양실조와 생식 성공률이 감소하여 북극곰 개체군이 더욱 위험에 처할 수 있습니다.

셋째, 기후 변화는 북극곰의 보금자리 구역에도 영향을 미치고 있습니다. 더 따뜻한 기온과 불안정한 눈 조건으로 인해 임신한 암컷이 적합한 보금자리 지역을 찾는 것이 더 어려워지고 있으며, 이는 새끼의 출생과 생존에 매우 중요합니다.

마지막으로, 전통적인 식량 공급원이 줄어들면서 일부 북극곰은 인간의 배설물이나 바다새와 그 알을 잡아먹는 행동을 보이고 있지만, 이는 고지방 식단을 대체하지 못해 장기적인 생존에 도움이 되지 않습니다. 이러한 여러 요인으로 인해 북극곰은 멸종 위기에 처할 수 있습니다.
